In [ ]:
-- Create dimension tables
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(255),
    brand VARCHAR(255)
    -- Add other product attributes
);

CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    address VARCHAR(255)
    -- Add other customer attributes
);

CREATE TABLE time (
    date DATE PRIMARY KEY,
    year INT,
    month INT,
    day INT,
    quarter INT
    -- Add other time attributes
);

-- Create fact table
CREATE TABLE sales (
    product_id INT,
    customer_id INT,
    date DATE,
    sales_amount DECIMAL(10, 2),
    -- Add other measures and foreign keys to additional dimensions
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (date) REFERENCES time(date)
);

-- Populate the fact table with sample data
INSERT INTO sales (product_id, customer_id, date, sales_amount)
VALUES
    (1, 1, '2022-01-01', 100.00),
    (2, 2, '2022-01-01', 50.00),
    -- Add more sample data
    ;


SELECT p.product_name, SUM(s.sales_amount) AS total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name;


SELECT t.year, t.quarter, SUM(s.sales_amount) AS total_sales
FROM sales s
JOIN time t ON s.date = t.date
GROUP BY t.year, t.quarter;


SELECT t.date, s.sales_amount
FROM sales s
JOIN time t ON s.date = t.date
WHERE s.product_id = 1
    AND t.date BETWEEN '2022-01-01' AND '2022-12-31';




In [ ]:
import csv
import psycopg2

# Extraction: Read data from CSV files
def extract_data(csv_file):
    extracted_data = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            extracted_data.append(row)
    return extracted_data

# Transformation: Apply business rules or calculations
def transform_data(data):
    transformed_data = []
    for row in data:
        # Apply transformations, e.g., calculate revenue based on quantity and price
        revenue = float(row['quantity']) * float(row['price'])
        row['revenue'] = revenue
        transformed_data.append(row)
    return transformed_data

# Loading: Load transformed data into a data warehouse
def load_data(data):
    conn = psycopg2.connect(
        host="localhost",
        database="mydatawarehouse",
        user="myuser",
        password="mypassword"
    )
    cursor = conn.cursor()
    
    for row in data:
        # Load data into the data warehouse table
        query = f"INSERT INTO mytable (product_id, quantity, price, revenue) VALUES ({row['product_id']}, {row['quantity']}, {row['price']}, {row['revenue']})"
        cursor.execute(query)
    
    conn.commit()
    cursor.close()
    conn.close()

# Example usage
csv_file = 'data.csv'  # Replace with your CSV file path

# Extract data from CSV
extracted_data = extract_data(csv_file)

# Transform data
transformed_data = transform_data(extracted_data)

# Load transformed data into the data warehouse
load_data(transformed_data)


In [ ]:
-- Create dimension tables
CREATE TABLE students (
    student_id SERIAL PRIMARY KEY,
    student_name VARCHAR(255),
    student_major VARCHAR(255)
    -- Add other student attributes
);

CREATE TABLE courses (
    course_id SERIAL PRIMARY KEY,
    course_name VARCHAR(255),
    course_department VARCHAR(255)
    -- Add other course attributes
);

CREATE TABLE time (
    time_id SERIAL PRIMARY KEY,
    date DATE,
    semester VARCHAR(255),
    year INT
    -- Add other time attributes
);

-- Create fact table
CREATE TABLE enrollments (
    enrollment_id SERIAL PRIMARY KEY,
    student_id INT,
    course_id INT,
    time_id INT,
    -- Add other measures and foreign keys to additional dimensions
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id),
    FOREIGN KEY (time_id) REFERENCES time(time_id)
);

-- Insert sample data into dimension tables
INSERT INTO students (student_name, student_major)
VALUES
    ('John Doe', 'Computer Science'),
    ('Jane Smith', 'Psychology'),
    -- Add more sample data
    ;

INSERT INTO courses (course_name, course_department)
VALUES
    ('Database Management', 'Computer Science'),
    ('Intro to Psychology', 'Psychology'),
    -- Add more sample data
    ;

INSERT INTO time (date, semester, year)
VALUES
    ('2022-01-01', 'Spring', 2022),
    ('2022-06-01', 'Summer', 2022),
    -- Add more sample data
    ;

-- Insert sample data into the fact table (enrollments)
INSERT INTO enrollments (student_id, course_id, time_id)
VALUES
    (1, 1, 1),
    (2, 2, 2),
    -- Add more sample data
    ;



SELECT t.semester, COUNT(*) AS total_enrollments
FROM enrollments e
JOIN time t ON e.time_id = t.time_id
GROUP BY t.semester;


SELECT s.student_name, c.course_name
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id;


SELECT c.course_department, COUNT(*) AS total_enrollments
FROM enrollments e
JOIN courses c ON e.course_id = c.course_id
GROUP BY c.course_department;




In [ ]:
import time
import csv
from multiprocessing import Pool
from pymongo import MongoClient

# Function to load data in bulk using batch processing
def load_data_batch(data):
    # Connect to the database
    client = MongoClient("mongodb://localhost:27017/")
    database = client["mydatawarehouse"]  # Replace with your database name
    collection = database["mycollection"]  # Replace with your collection name

    # Insert data in bulk
    collection.insert_many(data)

# Function to process a batch of data
def process_batch(batch):
    # Transform data if required
    processed_data = batch  # Placeholder, modify as per your transformation logic
    load_data_batch(processed_data)

# Function to load data using multi-threading or multiprocessing
def load_data_parallel(data, batch_size, num_processes):
    # Split data into batches
    batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

    # Create a pool of processes
    pool = Pool(processes=num_processes)

    # Start the parallel data loading process
    pool.map(process_batch, batches)

    # Close the pool
    pool.close()
    pool.join()

# Function to measure the time taken to load data
def measure_loading_time(data, batch_size, num_processes):
    start_time = time.time()

    # Call the optimized data loading function
    load_data_parallel(data, batch_size, num_processes)

    end_time = time.time()
    loading_time = end_time - start_time
    print(f"Data loading time: {loading_time} seconds")

# Example usage
csv_file = 'data.csv'  # Replace with your CSV file path
batch_size = 1000  # Number of records per batch
num_processes = 4  # Number of parallel processes

# Extract data from CSV
data = []
with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append(row)

# Measure loading time before optimizations
measure_loading_time(data, 1, 1)

# Measure loading time after optimizations
measure_loading_time(data, batch_size, num_processes)
